In [11]:

!pip install flask flask-cors pyngrok pandas matplotlib
!pkill ngrok

import time
import random
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
from flask import Flask, jsonify, send_from_directory
from flask_cors import CORS
from threading import Thread
from pyngrok import ngrok

# Initialize Flask
app = Flask(__name__)


CORS(app, resources={
    r"/*": {
        "origins": [
            "http://localhost:3000",
            "https://*.ngrok-free.app",

        ],
        "methods": ["GET", "POST", "OPTIONS"],
        "allow_headers": [
            "Content-Type",
            "User-Agent",
            "ngrok-skip-browser-warning",
            "Cache-Control"
        ],
        "supports_credentials": True
    }
})


#CORS(app, resources={r"/*": {"origins": "*"}})

# Ngrok Configuration
NGROK_AUTHTOKEN = "2sDUAt3lglQCWmGXxgFCcGTCeSa_3MN13k8g6y79iT7oc44MR"  # Replace with your token
ngrok.set_auth_token(NGROK_AUTHTOKEN)
# public_url = ngrok.connect(5001).public_url
# print(f"Ngrok URL: {public_url}")

# public_url = ngrok.connect(addr=5001, bind_tls=True).public_url

public_url = ngrok.connect(addr=5001, bind_tls=True).public_url.replace('http://', 'https://')
print(f"Ngrok URL: {public_url}")



def generate_smart_meter_data():
    return {
        "meter_id": "WaterMeter_001",
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "water_usage_liters": round(random.uniform(50, 200), 2),
        "battery_status": round(random.uniform(3.0, 4.2), 2),
    }

def encode_packet(data):
    data["checksum"] = sum(ord(c) for c in json.dumps(data)) % 256
    return data

def send_data_via_lora(packet):
    print(f"Simulating LoRa transmission: {packet}")

def receive_data_via_lora():
    data = generate_smart_meter_data()
    encoded_packet = encode_packet(data)
    print(f"Simulated received packet: {encoded_packet}")
    return encoded_packet

def decode_packet(packet):
    received_checksum = packet.pop("checksum", None)
    calculated_checksum = sum(ord(c) for c in json.dumps(packet)) % 256
    if received_checksum == calculated_checksum:
        print("Packet integrity verified.")
        return packet
    else:
        print("Packet integrity check failed.")
        return None

def log_received_data(data):
    try:
        with open('meter_data.log.txt', 'a') as file:
            file.write(json.dumps(data) + '\n')
        print(f"Data logged: {data}")
    except Exception as e:
        print(f"Error logging data: {e}")

def visualize_data():
    try:
        if not os.path.exists('meter_data.log.txt'):
            print("Log file not found.")
            return

        data = pd.read_json('meter_data.log.txt', lines=True)
        data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce', format='%Y-%m-%d %H:%M')
        data = data.dropna(subset=['timestamp'])
        data = data.sort_values(by='timestamp')

        plt.figure()
        plt.plot(data['timestamp'], data['water_usage_liters'], label='Water Usage (Liters) Over Time')
        plt.xlabel('Timestamp')
        plt.ylabel('Water Usage (Liters)')
        plt.title('Water Usage Over Time')
        plt.legend()
        plt.grid()

        image_path = '/content/drive/MyDrive/water_usage_graph_2.png'
        plt.savefig(image_path)
        plt.close()

        print("Visualization complete. Image saved as water_usage_graph_2.png.")
    except Exception as e:
        print(f"Error visualizing data: {e}")

# Flask Routes
@app.route('/')
def home():
    return jsonify({"message": "Smart Meter API"})

@app.route('/get-ngrok-url')
def get_ngrok_url():
    return jsonify({
        "ngrok_url": public_url.split('?')[0]  # Return URL without query params
    })


@app.route('/log-data', methods=['POST'])
def log_data():
    try:
        received_packet = receive_data_via_lora()
        decoded_data = decode_packet(received_packet)
        if decoded_data:
            log_received_data(decoded_data)
        return jsonify({"status": "success", "message": "Data received and logged."}), 200
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

@app.route('/visualize')
def visualize():
    try:
        visualize_data()
        return jsonify({"status": "success", "message": "Visualization complete."}), 200
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500


image_path = '/content/drive/MyDrive/water_usage_graph_2.png'  # Save path
@app.route('/images/<filename>')
def get_image(filename):
    try:
        return send_from_directory('/content/drive/MyDrive', filename)
    except Exception as e:
        return jsonify({"error": str(e)}), 500


# Main Execution
if __name__ == '__main__':
    from google.colab import drive
    drive.mount('/content/drive')

    def run_flask():
        app.run(host="0.0.0.0", port=5001)

    flask_thread = Thread(target=run_flask)
    flask_thread.start()

    try:
        print("Starting LoRa Simulation...")
        for _ in range(40):
            received_packet = receive_data_via_lora()
            decoded_data = decode_packet(received_packet)
            if decoded_data:
                log_received_data(decoded_data)
            time.sleep(2)

        print("\nVisualizing logged data...")
        visualize_data()
    except KeyboardInterrupt:
        print("Stopping the LoRa simulation.")


Ngrok URL: https://e883-34-139-237-33.ngrok-free.app
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting LoRa Simulation...
Simulated received packet: {'meter_id': 'WaterMeter_001', 'timestamp': '2025-01-30 10:42:14', 'water_usage_liters': 129.76, 'battery_status': 3.16, 'checksum': 22}
Packet integrity verified.
Data logged: {'meter_id': 'WaterMeter_001', 'timestamp': '2025-01-30 10:42:14', 'water_usage_liters': 129.76, 'battery_status': 3.16}
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5001 is in use by another program. Either identify and stop that program, or start the server with a different port.


Simulated received packet: {'meter_id': 'WaterMeter_001', 'timestamp': '2025-01-30 10:42:16', 'water_usage_liters': 115.34, 'battery_status': 3.43, 'checksum': 13}
Packet integrity verified.
Data logged: {'meter_id': 'WaterMeter_001', 'timestamp': '2025-01-30 10:42:16', 'water_usage_liters': 115.34, 'battery_status': 3.43}
Simulated received packet: {'meter_id': 'WaterMeter_001', 'timestamp': '2025-01-30 10:42:18', 'water_usage_liters': 60.81, 'battery_status': 3.02, 'checksum': 219}
Packet integrity verified.
Data logged: {'meter_id': 'WaterMeter_001', 'timestamp': '2025-01-30 10:42:18', 'water_usage_liters': 60.81, 'battery_status': 3.02}
Simulated received packet: {'meter_id': 'WaterMeter_001', 'timestamp': '2025-01-30 10:42:20', 'water_usage_liters': 148.98, 'battery_status': 4.16, 'checksum': 25}
Packet integrity verified.
Data logged: {'meter_id': 'WaterMeter_001', 'timestamp': '2025-01-30 10:42:20', 'water_usage_liters': 148.98, 'battery_status': 4.16}
Simulated received packet: